# Confidence and Choice Visualizer

Run DRAM_classify_blobs 10 times (remember to change model_name, then use ClassificationDistributions.ipynb to output confidence_one_run for each run, and set them as the confidence for each run in confidence_all_runs.

In [1]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [2]:
num_runs = 10
z_size = 9

confidence_all_runs = np.zeros([num_runs, z_size, z_size])
choice_all_runs = np.zeros([num_runs, z_size, z_size])

In [3]:
confidence_all_runs[0] = np.array([[0.860760747172113, 0.1277467767290509, 0.010657337000120606, 0.0007285194326832247, 6.787944351642578e-05, 2.9971955746236867e-05, 5.145775413146485e-06, 7.464326014394856e-07, 2.878851030397534e-06], [0.20300415922482953, 0.5184953242166103, 0.20827241095635282, 0.05231078128653731, 0.012505148318335242, 0.004580371038418926, 0.0006180559509162963, 0.00010845492016917556, 0.00010529498801776148], [0.017868306261641757, 0.3537408285439692, 0.34259592842755954, 0.1658108561628833, 0.07394838094437096, 0.037945567332524535, 0.0058128362575559, 0.0015080746380952204, 0.0007692225027735955], [0.0009324774553806627, 0.13202229804306542, 0.274546225576406, 0.23018302718121686, 0.16847032805212483, 0.14204805273613272, 0.030567274960408255, 0.014227405173598084, 0.007002910586340956], [0.0003520191396409994, 0.04111785734034918, 0.15463699939020453, 0.2050167920370368, 0.2134346593449809, 0.24783463010188098, 0.06838748303711407, 0.04378451394166884, 0.02543504510230652], [5.000795444690018e-06, 0.01044296698407568, 0.06641280716187803, 0.13162901687612752, 0.18505500357876825, 0.31054571315684004, 0.1161021573583348, 0.10390606812600003, 0.07590126688454743], [1.3284532939238279e-06, 0.0023274134316703856, 0.021358992187837482, 0.06635526982578113, 0.1281593052150456, 0.29800758347616646, 0.14616071119542176, 0.17792251735337322, 0.15970687958101304], [8.615321391135292e-08, 0.000515853959362386, 0.006578610663797049, 0.02975448711197077, 0.0733464273270929, 0.2329618283142827, 0.14965183055811213, 0.2368133880313326, 0.2703774875925387], [1.8369941338283736e-08, 0.00014249697360372787, 0.002083101829095979, 0.012640375018897986, 0.036423716315071956, 0.1483100470611913, 0.12314310943966848, 0.268005807373178, 0.4092513283179578]])
choice_all_runs[0] = np.array([[0.9582000000001493, 0.04180000000000032, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.2021999999999928, 0.6375000000000764, 0.15339999999999557, 0.006099999999999994, 0.0002, 0.0006, 0.0, 0.0, 0.0], [0.008399999999999986, 0.3843000000000189, 0.4561000000000352, 0.10799999999999814, 0.029400000000000148, 0.013699999999999967, 0.0001, 0.0, 0.0], [0.0004, 0.08969999999999918, 0.39560000000002143, 0.2231999999999916, 0.10709999999999818, 0.18179999999999397, 0.00030000000000000003, 0.0013, 0.0006], [0.0002, 0.013699999999999967, 0.18319999999999387, 0.21249999999999222, 0.1493999999999958, 0.41790000000002653, 0.003900000000000002, 0.011399999999999976, 0.007799999999999988], [0.0, 0.0007999999999999999, 0.056500000000000536, 0.10939999999999805, 0.10209999999999847, 0.6116000000000705, 0.008899999999999984, 0.05640000000000053, 0.0543000000000005], [0.0, 0.0007999999999999999, 0.007599999999999989, 0.03770000000000027, 0.05780000000000055, 0.5834000000000641, 0.013599999999999968, 0.13329999999999673, 0.16579999999999487], [0.0, 0.0002, 0.0014000000000000002, 0.007499999999999989, 0.018799999999999997, 0.41000000000002473, 0.015299999999999961, 0.18219999999999392, 0.3646000000000144], [0.0, 0.0, 0.0004, 0.0022000000000000006, 0.005999999999999995, 0.21529999999999205, 0.012399999999999972, 0.15059999999999574, 0.6131000000000708]])

In [5]:
yo = confidence_all_runs[0]
data = [go.Heatmap(
    z=yo,# - human_confusion,
    colorscale="Viridis"
)]

layout = go.Layout(
    title="DRAM_classify_blobs Confidence",
    yaxis=dict(
        autorange='reversed',
        title="True Class",
        dtick=1
    ),
    xaxis=dict(
        title="Predicted Class",
        dtick=1
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [6]:
yo = choice_all_runs[0]
data = [go.Heatmap(
    z=yo,# - human_confusion,
    colorscale="Viridis"
)]

layout = go.Layout(
    title="DRAM_classify_blobs Choice",
    yaxis=dict(
        autorange='reversed',
        title="True Class",
        dtick=1
    ),
    xaxis=dict(
        title="Predicted Class",
        dtick=1
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)